In [1]:
import os
import yaml
import openai
from openai import OpenAI

client = OpenAI()

## 1. Define the function

In [2]:
import json

def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""

    weather_info = {
        "location": location,
        "unit": unit,
        "temperature": 20,
        "description": "sunny",
    }

    return json.dumps(weather_info)


functions = [
    {
        'name': 'get_current_weather',
        'description': 'Get the current weather in a given location',
        'parameters': {
            'type': 'object',
            'properties': {
                'location': {
                    'type': 'string',
                    'description': "The city and state, e.g. San Francisco, CA"
                },
                'unit': {
                    'type': 'string',
                    'enum': ['celsius', 'fahrenheit']
                }
            },
            'required': ['location']
        }
    }
]

## 2. Relevant message

In [3]:
message = [
    {
        'role': 'user',
        'content': "너 이름이 뭐야?"
    }
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=message,
)

response.choices[0].message.content

'저는 AI 언어 모델로, 특정한 이름은 없습니다. 필요하신 질문이나 도움이 필요하신 부분이 있다면 언제든지 말씀해 주세요!'

In [4]:
message = [
    {
        'role': 'user',
        'content': "How's the weather in Boston?"
    }
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=message,
    functions=functions
)

response.choices[0].message # function_call is not None


ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"Boston, MA"}', name='get_current_weather'), tool_calls=None)

In [5]:
response.choices[0].message.function_call.arguments

'{"location":"Boston, MA"}'

In [6]:
eval(response.choices[0].message.function_call.arguments)

{'location': 'Boston, MA'}

In [7]:
get_current_weather(eval(response.choices[0].message.function_call.arguments))

'{"location": {"location": "Boston, MA"}, "unit": "fahrenheit", "temperature": 20, "description": "sunny"}'

## 3. Unrelevant message

In [8]:
message = [
    {
        'role': 'user',
        'content': "hi!"
    }
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=message,
    functions=functions
)

response.choices[0].message # function_call is None

ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)

## 4. Forcing to use function calling

In [15]:
message = [
    {
        'role': 'user',
        'content': "hi!"
    }
]

response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=message,
    functions=functions,
    function_call={'name': 'get_current_weather'}
)

response.choices[0].message

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"New York, NY"}', name='get_current_weather'), tool_calls=None)

## 5. Forcing not to use function calling

In [14]:
message = [
    {
        'role': 'user',
        'content': "What's the weather like in Boston?"
    }
]

response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=message,
    functions=functions,
    function_call='none',
)

response.choices[0].message

ChatCompletionMessage(content='Please hold on for a moment while I get the current weather information for Boston.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)

## 6. Prompt size

In [16]:
message = [
    {
        'role': 'user',
        'content': "What's the weather like in Boston?"
    }
]

response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=message,
)

response.usage

CompletionUsage(completion_tokens=31, prompt_tokens=14, total_tokens=45, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

In [18]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=message,
    functions=functions,
)

response.usage

CompletionUsage(completion_tokens=18, prompt_tokens=79, total_tokens=97, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

In [21]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=message,
    functions=functions,
    function_call={'name': 'get_current_weather'}
)

response.usage

CompletionUsage(completion_tokens=8, prompt_tokens=89, total_tokens=97, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

In [20]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=message,
    functions=functions,
    function_call='none',
)

response.usage

CompletionUsage(completion_tokens=16, prompt_tokens=80, total_tokens=96, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

## 7. NLG using the function calling result

In [23]:
message = [
    {
        'role': 'user',
        'content': "What's the weather like in Boston?"
    }
]

response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=message,
    functions=functions,
    function_call={'name': 'get_current_weather'}
)


In [24]:
response_message = response.choices[0].message
response_message

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"Boston, MA"}', name='get_current_weather'), tool_calls=None)

In [25]:
message.append(response_message)
message

[{'role': 'user', 'content': "What's the weather like in Boston?"},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"Boston, MA"}', name='get_current_weather'), tool_calls=None)]

In [26]:
args = eval(response_message.function_call.arguments)
args

{'location': 'Boston, MA'}

In [27]:
observation = get_current_weather(**args)
observation

'{"location": "Boston, MA", "unit": "fahrenheit", "temperature": 20, "description": "sunny"}'

In [28]:
message.append(
    {
        'role': 'function',
        'name': 'get_current_weather',
        'content': observation
    }
)

message

[{'role': 'user', 'content': "What's the weather like in Boston?"},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"Boston, MA"}', name='get_current_weather'), tool_calls=None),
 {'role': 'function',
  'name': 'get_current_weather',
  'content': '{"location": "Boston, MA", "unit": "fahrenheit", "temperature": 20, "description": "sunny"}'}]

In [29]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=message,
)

response.choices[0].message.content

'The current weather in Boston is sunny, with a temperature of 20°F.'